In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/7.vehicle'):
    shutil.rmtree('./file/7.vehicle')

os.makedirs('./file/7.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/6.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/6.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    try:
                        resp = requests.get(f'''https://www.drivparts.com/driv/partfinder/api.catalog.applications?brand=corporate&locale=en_US&country_code=US&part_number={input_.loc[a, 'Part Number']}&brand_code={input_.loc[a, 'Brand']}''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            json_ = resp.json()

                            list_vehicle = json_['application_group_list']['PASSENGER CAR & LIGHT TRUCK']['applications']
                            if list_vehicle:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for vehicle in list_vehicle:                        
                    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                 'Url': input_.loc[a, 'Url'],
                                                 'Make': vehicle['make']['value'].strip(),
                                                 'Model': vehicle['model']['value'].strip(),
                                                 'Engine': vehicle['engine_base']['engine_base_value'].strip() if 'engine_base' in vehicle else vehicle['engine_base_value'].strip(),
                                                 'Engine VIN': vehicle['engine_vin']['value'].strip() if 'engine_vin' in vehicle else '',
                                                 'Engine Desg.': vehicle['engine_designation']['value'].strip() if 'engine_designation' in vehicle else '',
                                                 'Year': vehicle['years'],
                                                 'Description': vehicle['category']['part_type_value'].strip(),
                                                 'Position': '' if 'position' not in vehicle or vehicle['position']['value'].strip() == 'N/A' else vehicle['position']['value'].strip(),
                                                 'Drive Wheel': vehicle['drive']['value'].strip() if 'drive' in vehicle else '',
                                                 'Veh. Qty.': vehicle['qty']})
                    
                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(30):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make', 'Model', 'Engine', 'Year'],
                                                    ascending=[True, True, True, True, False],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/7.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/7.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/7.vehicle/{file}',
              f'''./file/7.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：34] - 100.00% - type_99.Tie Rod End Bushing Kit > 7613.EV119
[剩余数量：0] - [当前时间：15:40:52]

输出ing...



Progress: 100%|██████████████████████████| 115/115 [00:00<00:00, 5947.68it/s]

Done ~
